<a href="https://colab.research.google.com/github/userlee123/Hypothesis-testing-Python/blob/main/Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    uni_df = pd.DataFrame(data=None, columns = ["State", "RegionName"])
    file1 = open('university_towns.txt', 'r') 
    Lines = file1.readlines()
    Stname = ""
    for l in Lines:
        Rname = ""
        app = 0
        if( (l.find('[',0, len(l)) > 0 ) | ( l.find('(', 0, len(l)) > 0) ):
            for i in range ( 0,len(l) - 1):
                if l[i] == '[':
                    Stname = l[:i]
                    break
                elif ( l[i] == ' ' ) & ( l[ i+1 ] == '(' ):
                    Rname = l[:i]
                    app = 1
                    break
        else:
            for j in range ( 0,len(l) ):
                if l[j] == ':':
                    Rname = l[:j]
                    app=1
                if l[j] == '\n':
                    Rname = l[:j]
                    app = 1
        if (app == 1):
            df2 = pd.DataFrame({"State": [Stname],
                               "RegionName": [Rname]})
            uni_df = uni_df.append(df2, ignore_index = True)
            uni_df = uni_df.reindex(columns=["State", "RegionName"])
    return uni_df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows = 7)
    gdp.columns = ['1', '2', '3', '4', 'quarter', '6', 'gdp', '8']
    gdp = gdp[['quarter','gdp']]
    for item in gdp[gdp['quarter'] == '2000q1'].index:
        skip = item
    gdp = gdp.loc[skip:]
    gdp['change'] = gdp['gdp'].diff()
    gdp2 = gdp[gdp['change'] < 0]
    rec = []
    rec2 = []
    for i in gdp2.index:
        rec.append(i)
        rec2.append(i)
    n = len(rec)
    for j in range(0,n - 1):
        if (rec[j] - rec[j+1] == -1):
            rec2.remove(rec[j+1])
        elif((j > 0) & (rec[j] - rec[j+1] != -1) & (rec[j] - rec[j - 1] != 1)):
            rec2.remove(rec[j])
    rec2.remove(rec2[0])
    rec2.remove(rec2[-1])
    gdpRec=gdp2.loc[rec2]
    return gdpRec.iloc[0]['quarter']
get_recession_start()

'2008q3'

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows = 7)
    gdp.columns = ['1', '2', '3', '4', 'quarter', '6', 'gdp', '8']
    gdp = gdp[['quarter','gdp']]
    for item in gdp[gdp['quarter'] == get_recession_start() ].index:
        skip = item
    gdp = gdp.loc[skip:]
    gdp['change'] = gdp['gdp'].diff()
    gdp2 = gdp[gdp['change'] > 0]
    grow2 = []
    for i in gdp2.index:
        grow2.append(i)
    for j in range(len(grow2)):
        if grow2[j] - grow2[j+1] == -1:
            end = grow2[j + 1]
            break
    gdpEnd = gdp2.loc[end]   
    return gdpEnd.iloc[0]
get_recession_end()

'2009q4'

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows = 7)
    gdp.columns = ['1', '2', '3', '4', 'quarter', '6', 'gdp', '8']
    gdp = gdp[['quarter','gdp']]
    for item in gdp[gdp['quarter'] == get_recession_start() ].index:
        skip1 = item
    for item in gdp[gdp['quarter'] == get_recession_end() ].index:
        skip2 = item
    gdp = gdp.loc[skip1:skip2]
    gdpMin = np.min(gdp['gdp'])
    btm = gdp[gdp['gdp'] == gdpMin]
    return btm.iloc[0]['quarter']
get_recession_bottom()

'2009q2'

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house = pd.read_csv('City_Zhvi_AllHomes.csv')
    def setFull(row):
        data = row['State']
        row['State'] = states.get(data)
        return row
    house = house.apply(setFull, axis = 1)
    house = house.set_index(["State","RegionName"])
    house2 = house.iloc[:, 49:249]
    house2.columns = pd.to_datetime(house2.columns)
    house2 = house2.resample('Q',axis=1).mean()
    yr = 2000
    qtr = 1
    rename = []
    for i in range(0, len(house2.columns)):
        name = str(yr) + 'q' + str(qtr)
        if (qtr == 4):
            yr+=1
            qtr=0
        qtr+=1
        rename.append(name)
    house2.columns = rename
    return house2

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    gdp = pd.read_excel('gdplev.xls',skiprows = 7)
    gdp.columns = ['1', '2', '3', '4', 'quarter', '6', 'gdp', '8']
    gdp = gdp[['quarter','gdp']]
    for item in gdp[gdp['quarter'] == get_recession_start() ].index:
            b4recIndex = item - 1
    b4rec = gdp.iloc[b4recIndex]['quarter']
    price = convert_housing_data_to_quarters()
    price['price_ratio'] = price[b4rec]/price[get_recession_bottom()]
    uni = get_list_of_university_towns()
    uniP = pd.merge(price, uni, how = "inner", left_index=True, right_on = ["State","RegionName"])
    uniP = uniP.set_index(["State","RegionName"])
    nonP = price.drop(uniP.index)
    p = ttest_ind(uniP['price_ratio'], nonP['price_ratio'], nan_policy="omit").pvalue
    if p < 0.01 :
        different = True
    else:
        different = False
    if (np.mean(uniP['price_ratio']) < np.mean(nonP['price_ratio']) ):
        better = "university town"
    else:
        better = "non-university town"
    result = (different, p, better)
    return result

(True, 0.0027240637047531249, 'university town')